<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/tfrs_context_features_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# TFRS Recreate tutorial - Taking advantage of context features



In [the featurization tutorial](featurization) we incorporated multiple features beyond just user and movie identifiers into our models, but we haven't explored whether those features improve model accuracy.

Many factors affect whether features beyond ids are useful in a recommender model:

1. __Importance of context__: if user preferences are relatively stable across contexts and time, context features may not provide much benefit. If, however, users preferences are highly contextual, adding context will improve the model significantly. For example, day of the week may be an important feature when deciding whether to recommend a short clip or a movie: users may only have time to watch short content during the week, but can relax and enjoy a full-length movie during the weekend. Similarly, query timestamps may play an important role in modelling popularity dynamics: one movie may be highly popular around the time of its release, but decay quickly afterwards. Conversely, other movies may be evergreens that are happily watched time and time again.
2. __Data sparsity__: using non-id features may be critical if data is sparse. With few observations available for a given user or item, the model may struggle with estimating a good per-user or per-item representation. To build an accurate model, other features such as item categories, descriptions, and images have to be used to help the model generalize beyond the training data. This is especially relevant in [cold-start](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)) situations, where relatively little data is available on some items or users.

In this tutorial, we'll experiment with using features beyond movie titles and user ids to our MovieLens model.

## Preliminaries

We first import the necessary packages.

In [1]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.1 MB/s eta 0:00:00


In [2]:
import os
import tempfile

import numpy as np
import tensorflow as tf


import tensorflow_recommenders as tfrs

We follow [the featurization tutorial](featurization) and keep the user id, timestamp, and movie title features.

In [6]:
ratings_data = pd.read_csv("/content/mock-data_interaction.csv", index_col = 0)
recipes_data = pd.read_csv("/content/mock-data_recipe.csv", index_col = 0).drop(columns = ["nutritions", "image_url"])

ratings_data.dateLastModified = pd.to_datetime(ratings_data.dateLastModified).astype(int)



In [39]:
ratings_data = ratings_data.merge(
    recipes_data[['recipe_id', 'recipe_name']],
    on='recipe_id',
    how='left'
)

In [43]:
ratings_data.head(5)

,user_id,recipe_id,rating,dateLastModified,recipe_name
0,8542392,222388,5,1492865203663000000,Homemade Bacon
1,11174581,222388,5,1371743425960000000,Homemade Bacon
2,8262477,222388,5,1423898871307000000,Homemade Bacon
3,3574785,240488,5,1507400408973000000,"Pork Loin, Apples, and Sauerkraut"
4,12145410,240488,2,1515197169563000000,"Pork Loin, Apples, and Sauerkraut"


In [44]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data))
recipes = tf.data.Dataset.from_tensor_slices(dict(recipes_data))

In [45]:
# ratings = tfds.load("movielens/100k-ratings", split="train")
# movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"],
    "dateLastModified": x["dateLastModified"],
    "recipe_name": x["recipe_name"],

})
recipes = recipes.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "recipe_name": x["recipe_name"],
})

We also do some housekeeping to prepare feature vocabularies.

In [46]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["dateLastModified"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_recipe_ids = np.unique(np.concatenate(list(recipes.batch(1000).map(
    lambda x: x["recipe_id"]
))))

unique_recipe_names = np.unique(np.concatenate(list(recipes.batch(1000).map(
    lambda x: x["recipe_name"]
))))

unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))



## Model definition

### Query model

We start with the user model defined in [the featurization tutorial](featurization) as the first layer of our model, tasked with converting raw input examples into feature embeddings. However, we change it slightly to allow us to turn timestamp features on or off. This will allow us to more easily demonstrate the effect that timestamp features have on the model. In the code below, the `use_timestamps` parameter gives us control over whether we use timestamp features.

In [47]:
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.IntegerLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["dateLastModified"]),
        tf.reshape(self.normalized_timestamp(inputs["dateLastModified"]), (-1, 1)),
    ], axis=1)

Note that our use of timestamp features in this tutorial interacts with our choice of training-test split in an undesirable way. Because we have split our data randomly rather than chronologically (to ensure that events that belong to the test dataset happen later than those in the training set), our model can effectively learn from the future. This is unrealistic: after all, we cannot train a model today on data from tomorrow.

This means that adding time features to the model lets it learn _future_ interaction patterns. We do this for illustration purposes only: the MovieLens dataset itself is very dense, and unlike many real-world datasets does not benefit greatly from features beyond user ids and movie titles.

This caveat aside, real-world models may well benefit from other time-based features such as time of day or day of the week, especially if the data has strong seasonal patterns.

### Candidate model

For simplicity, we'll keep the candidate model fixed. Again, we copy it from the [featurization](featurization) tutorial:

In [48]:
class RecipeModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.id_embedding = tf.keras.Sequential([
      tf.keras.layers.IntegerLookup(
          vocabulary=unique_recipe_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_recipe_ids) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(unique_recipe_names)

  def call(self, inputs):
    return tf.concat([
        self.id_embedding(inputs["recipe_id"]),
        self.title_text_embedding(inputs["recipe_name"]),
    ], axis=1)

### Combined model

With both `UserModel` and `MovieModel` defined, we can put together a combined model and implement our loss and metrics logic.

Here we're building a retrieval model. For a refresher on how this works, see the [Basic retrieval](basic_retrieval.ipynb) tutorial.

Note that we also need to make sure that the query model and candidate model output embeddings of compatible size. Because we'll be varying their sizes by adding more features, the easiest way to accomplish this is to use a dense projection layer after each model:



In [49]:
class RecipeRecommedModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      RecipeModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=recipes.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "dateLastModified": features["dateLastModified"],
    })
    recipe_embeddings = self.candidate_model({
        "recipe_id": features["recipe_id"],
        "recipe_name": features["recipe_name"],
    })

    return self.task(query_embeddings, recipe_embeddings)

## Experiments

### Prepare the data

We first split the data into a training set and a testing set.

In [50]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(ratings.cardinality(), seed=42, reshuffle_each_iteration=False)

train = shuffled.take(6_000)
test = shuffled.skip(6_000).take(2_000)

cached_train = train.shuffle(6_000).batch(1024)
cached_test = test.batch(512).cache()

In [51]:
2**9

512

### Baseline: no timestamp features

We're ready to try out our first model: let's start with not using timestamp features to establish our baseline.

In [52]:
model = RecipeRecommedModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
6/6 [==============================] - 3s 112ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0407 - factorized_top_k/top_5_categorical_accuracy: 0.1533 - factorized_top_k/top_10_categorical_accuracy: 0.2145 - factorized_top_k/top_50_categorical_accuracy: 0.4988 - factorized_top_k/top_100_categorical_accuracy: 0.9383 - loss: 6775.4819 - regularization_loss: 0.0000e+00 - total_loss: 6775.4819
Epoch 2/3
6/6 [==============================] - 1s 106ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1513 - factorized_top_k/top_5_categorical_accuracy: 0.3460 - factorized_top_k/top_10_categorical_accuracy: 0.4518 - factorized_top_k/top_50_categorical_accuracy: 0.7763 - factorized_top_k/top_100_categorical_accuracy: 0.9658 - loss: 6493.9994 - regularization_loss: 0.0000e+00 - total_loss: 6493.9994
Epoch 3/3
4/4 [==============================] - 1s 93ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0090 - factorized_top_k/top_5_categorical_accuracy: 0.0480 - fa

This gives us a baseline top-100 accuracy of around 0.2.



### Capturing time dynamics with time features

Do the result change if we add time features?

In [54]:
model = RecipeRecommedModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
6/6 [==============================] - 2s 110ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1173 - factorized_top_k/top_5_categorical_accuracy: 0.1593 - factorized_top_k/top_10_categorical_accuracy: 0.1890 - factorized_top_k/top_50_categorical_accuracy: 0.6253 - factorized_top_k/top_100_categorical_accuracy: 0.9567 - loss: 6928.2943 - regularization_loss: 0.0000e+00 - total_loss: 6928.2943
Epoch 2/3
6/6 [==============================] - 1s 112ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0807 - factorized_top_k/top_5_categorical_accuracy: 0.2418 - factorized_top_k/top_10_categorical_accuracy: 0.3657 - factorized_top_k/top_50_categorical_accuracy: 0.8810 - factorized_top_k/top_100_categorical_accuracy: 0.9983 - loss: 6254.6097 - regularization_loss: 0.0000e+00 - total_loss: 6254.6097
Epoch 3/3
4/4 [==============================] - 0s 101ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0500 - factorized_top_k/top_5_categorical_accuracy: 0.1585 - f

This is quite a bit better: not only is the training accuracy much higher, but the test accuracy is also substantially improved.

## Next Steps

This tutorial shows that even simple models can become more accurate when incorporating more features. However, to get the most of your features it's often necessary to build larger, deeper models. Have a look at the [deep retrieval tutorial](deep_recommenders) to explore this in more detail.